In [1]:
import numpy as np
import matplotlib as plt
import os
from itertools import repeat
from datetime import datetime
import shutil

In [63]:
# Pull in star logs and pull out obs number, target, UT time, observation type for each observation

currdir = os.getcwd()
stardir = currdir #+ #'/logs_2020'
os.chdir(stardir)

starlog_file = '2021-1028-all.txt'
obslog_file = '2021-1028.obslog.sent.txt'
obslist_file = '2021-10-26.obs.txt'
    

In [64]:
# Parse obslist
with open(obslist_file, 'r') as f:
    content = f.read().split('-\n!')[2].split('\n')[1:-1]
    content = [content[i] for i in range(len(content)) if content[i] != '']
    for i in range(len(content)):
        content[i] = [content[i].split()[0], content[i].split()[-13]]

    content_dict = {content[i][0]:content[i][1] for i in range(len(content))}
    print(content_dict)

{'BSC7447': 'C', 'FKV0749': '.', 'FKV0768': 'C', 'FKV0800': 'B', 'FKV0834': 'C', 'FKV0842': '.', 'FKV0850': 'C', 'FKV0855': '.', 'FKV0902': 'C', 'FKV0007': '.', 'FKV0085': 'C', 'FKV0066': 'B', 'FKV0151': 'C', 'FKV0123': 'B', 'FKV0169': 'C', 'FKV0154': '.', 'FKV0195': 'C', 'FKV0201': '.', 'FKV0219': 'C', 'FKV0220': '.', 'FKV0285': 'C', 'FKV1187': '.'}


In [72]:
def import_logs(starlog):
        # Begin data import
        data = []

        # Pull in starlog data, skipping the header. Should work even for starlogs
        # with multiple headers.
        
        with open(starlog, 'r') as f:
            content = f.read().split('\nUT_date: ')[1:]
            for c in content:
                csplit = c.split('\n')
                if csplit[17::2] != ['']:
                    data.append([csplit[17::2],csplit[1][10:]])       
                    
        # Combine all the starlogs into one list of entries, then split each entry
        # which is a single string containing the entire entry into a list of the individual column entries
        for i in range(len(data)):
            data[i][0] = [data[i][0][j].split() for j in range(len(data[i][0]))]

            for j in range(len(data)):
                if len(data[j][0]) == 0:
                    pass
                elif data[j][0][-1] == []:
                    data[j][0] = data[j][0][0:-1]
        
        for i in range(len(data)):
            for j in range(len(data[i][0])):
                if len(data[i][0][j][1]) != 1:
                    data[i][0][j].insert(1, 'X')
                data[i][0][j] = data[i][0][j][0:3] + data[i][0][j][15:]
        
        return data

In [73]:
import_logs(starlog_file)

[[[['1', 'C', 'FKV0834', '6-6', '(30)'],
   ['2', 'I', 'FKV0834'],
   ['3', 'I', 'FKV0834', 'E6', 'AE'],
   ['4', 'I', 'FKV0834', 'AC'],
   ['5', 'I', 'FKV0834', 'W4', 'W7'],
   ['6', 'I', 'FKV0834', 'AW'],
   ['7', '.', 'FKV0842', '6-6', '(15)'],
   ['8', 'I', 'FKV0842'],
   ['9', 'I', 'FKV0842', 'AE', 'E6'],
   ['10', 'I', 'FKV0842', 'AC'],
   ['11', 'I', 'FKV0842', 'W4', 'W7'],
   ['12', 'I', 'FKV0842', 'AW'],
   ['13', '.', 'FKV0842', '6-4', '(30)'],
   ['14', 'I', 'FKV0842'],
   ['15', 'I', 'FKV0902', 'AE', 'E6'],
   ['16', 'I', 'FKV0902', 'AC'],
   ['17', 'I', 'FKV0902', 'W4', 'W7'],
   ['18', 'I', 'FKV0902', 'AW'],
   ['19', '.', 'FKV0007', '6-6', '(30)'],
   ['20', 'I', 'FKV0007'],
   ['21', 'I', 'FKV0007', 'AE', 'E6'],
   ['22', 'I', 'FKV0007', 'AC'],
   ['23', 'I', 'FKV0007', 'W4', 'W7'],
   ['24', 'I', 'FKV0007', 'AW'],
   ['25', 'C', 'FKV0902', '6-6', '(30)'],
   ['26', 'I', 'FKV0902'],
   ['27', 'C', 'FKV0902', '6-6', '(30)'],
   ['28', 'I', 'FKV0902'],
   ['29', '.', 'FKV

In [20]:
# Write observation data to file, including observer and some basic analysis (total scans, time observing,
# average scans per hour, etc)
# Also can write number of observations per obstype and per star if desired

stat_file = open('observing_stats_2020.txt', 'a+')
stat_file.write('UT Date\tObserver\tTotal Scans\tTime Observing\tAverage Scans per Hour\n')

for i in range(len(starlog_files)):
    test = import_logs(starlog_files[i])

    for j in range(len(test)):
        
        if test[j][1] == '':
            test[j][1] = 'XXXX'
    
        stat_data = {'obsnum': [test[j][0][i][0] for i in range(len(test[j][0]))],
                    'obstype': [test[j][0][i][1] for i in range(len(test[j][0]))],
                       'star': [test[j][0][i][2] for i in range(len(test[j][0]))],
                     'uttime': [test[j][0][i][3] for i in range(len(test[j][0]))]}
    
        FMT = '%H:%M:%S'
        if len(stat_data['uttime']) >= 2:
            tdelta = (datetime.strptime(stat_data['uttime'][-1], FMT) - datetime.strptime(stat_data['uttime'][0], FMT)).seconds/3600
            obsperhour = round(len(test[j][0])/tdelta,1)
        else:
            tdelta = 0
            obsperhour = 0

        obscounts = {i: stat_data['obstype'].count(i) for i in list(set(stat_data['obstype']))}
        starcounts = {i: stat_data['star'].count(i) for i in list(set(stat_data['star']))}
    
        try:
            stat_file.write(starlog_files[i][0:9] + '\t' + test[j][1] + '\t' + str(len(test[j][0])) + '\t' + str(round(tdelta,1)) + '\t' +  str(obsperhour) + '\n')
#             for i in range(len(list(obscounts.keys()))):
#                 stat_file.write(list(obscounts.keys())[i] + '\t' + str(list(obscounts.values())[i]) + '\n')
#             for i in range(len(list(starcounts.keys()))):
#                 stat_file.write(list(starcounts.keys())[i] + '\t' + str(list(starcounts.values())[i]) + '\n')
        except IndexError:
            pass

stat_file.close()

In [9]:
# Code I used to combine data from different starlog files taken on the same night. Made analysis much easier.

# logsample = starlog_files

In [10]:
# for i in range(len(logsample)):
#     if i == 0:
#         fc = open(logsample[i][0:9] + '-all.txt', 'wb')
#         f = open(logsample[0], 'rb')
#         shutil.copyfileobj(f,fc)
#         f.close()       
#     else:
#         if logsample[i][0:9] == logsample[i-1][0:9]:
#             f = open(logsample[i], 'rb')
#             shutil.copyfileobj(f,fc)
#             f.close()
#         else:
#             fc.close()
#             fc = open(logsample[i][0:9] + '-all.txt', 'wb')
#             f = open(logsample[i], 'rb')
#             shutil.copyfileobj(f,fc)
#             f.close()

In [11]:
# # Function to pull out information about seeing from the obslog summary. UNDER DEVELOPMENT.
# def seeing(obslog):
#     # Begin data import
#     data = []

#     with open(obslog, 'r') as f:
#         content = f.read().split(']: Weather')
# #             for c in content:
# #                 csplit = c.split('\n')
# #                 #print(csplit[1][10:])
# #                 if csplit[17::2] != ['']:
# #                     data.append([csplit[17::2],csplit[1][10:]])

#     f.close()
#     return content

In [12]:
# for log in obslog_files:
#     txt = seeing(log)[-1]
#     print(log)
#     print([sentence + '.' for sentence in txt.split('.') if 'eeing' in sentence])




In [9]:
# Find the way of an obslist based on the number of stations it is configured for (ie no 'X' in the station list)
def way(obslist):
    # Begin data import
    data = []


    # Pull in starlog data, skipping the header. Should work even for starlogs
    # with multiple headers.

    with open(obslist, 'r') as f:
        content = f.read().split('Stations: ')
#             for c in content:
#                 csplit = c.split('\n')
#                 #print(csplit[1][10:])
#                 if csplit[17::2] != ['']:
#                     data.append([csplit[17::2],csplit[1][10:]])

    f.close()
    return content

In [10]:
for obslist in obslist_files[0:1]:
    stations = way(obslist)[1].split('\n')[0]
    theway = 6 - stations.count('X')
    print(stations)
    print(theway)

 E6(1)  AC(2)  AE(3)  W4(4)  AW(5)  W7(6)
6


In [20]:
obslist_files.sort()

In [21]:
# Basic sorting of starlogs by obslist based on what was the most recent obslist issued before the starlog date
# UNDER DEVELOPMENT
logs_by_list = {obslist_files[i][0:10]: [] for i in range(len(obslist_files))}

for i in range(len(obslist_files)):
    obs_min = datetime.strptime(obslist_files[i][0:10], '%Y-%m-%d')
    try:
        obs_max = datetime.strptime(obslist_files[i+1][0:10], '%Y-%m-%d')
    except IndexError:
        obs_max = datetime.now()
    #print('min: ' + str(obs_min))
    #print('max: ' + str(obs_max))
    for j in range(len(starlog_files)):
        logdate = datetime.strptime(starlog_files[j][0:9], '%Y-%m%d')
        if obs_min <= logdate < obs_max:
            #print(True)
            #print('starlog: ' + str(logdate))
            logs_by_list[obslist_files[i][0:10]].append(starlog_files[j])
        

In [22]:
logs_by_list

{'2020-10-28': ['2020-1028-all.txt'],
 '2020-10-30': ['2020-1102-all.txt',
  '2020-1103-all.txt',
  '2020-1104-all.txt',
  '2020-1105-all.txt'],
 '2020-11-10': ['2020-1110-all.txt'],
 '2020-12-04': ['2020-1209-all.txt', '2020-1210-all.txt'],
 '2020-12-15': ['2020-1216-all.txt', '2020-1217-all.txt'],
 '2020-12-21': ['2020-1221-all.txt',
  '2020-1222-all.txt',
  '2020-1223-all.txt',
  '2020-1229-all.txt',
  '2020-1230-all.txt',
  '2020-1231-all.txt',
  '2021-0106-all.txt'],
 '2021-01-07': ['2021-0107-all.txt',
  '2021-0109-all.txt',
  '2021-0112-all.txt',
  '2021-0113-all.txt',
  '2021-0114-all.txt',
  '2021-0115-all.txt',
  '2021-0128-all.txt',
  '2021-0129-all.txt',
  '2021-0202-all.txt'],
 '2021-02-03': ['2021-0203-all.txt',
  '2021-0204-all.txt',
  '2021-0205-all.txt',
  '2021-0210-all.txt',
  '2021-0211-all.txt',
  '2021-0212-all.txt',
  '2021-0213-all.txt',
  '2021-0214-all.txt',
  '2021-0304-all.txt'],
 '2021-03-05': ['2021-0305-all.txt',
  '2021-0306-all.txt',
  '2021-0307-all.tx

In [19]:
# Write observation information to file, now including the associated obslist and way
# UNDER DEVELOPMENT

global way_stat_file
way_stat_file = open('way_observing_stats_2020.txt', 'a+')
way_stat_file.write('Obslist\tWay\tUT Date\tObserver\tTotal Scans\tTime Observing\tAverage Scans per Hour\n')
way_stat_file.close()

def way_stat(obslist, starfiles, myfile, way_int):
    for k in range(len(starfiles)):
        test = import_logs(starfiles[k])
        
        for j in range(len(test)):

            if test[j][1] == '':
                test[j][1] = 'XXXX'

            stat_data = {'obsnum': [test[j][0][i][0] for i in range(len(test[j][0]))],
                        'obstype': [test[j][0][i][1] for i in range(len(test[j][0]))],
                           'star': [test[j][0][i][2] for i in range(len(test[j][0]))],
                         'uttime': [test[j][0][i][3] for i in range(len(test[j][0]))]}

            FMT = '%H:%M:%S'
            if len(stat_data['uttime']) >= 2:
                tdelta = (datetime.strptime(stat_data['uttime'][-1], FMT) - datetime.strptime(stat_data['uttime'][0], FMT)).seconds/3600
                obsperhour = round(len(test[j][0])/tdelta,1)
            else:
                tdelta = 0
                obsperhour = 0

            obscounts = {i: stat_data['obstype'].count(i) for i in list(set(stat_data['obstype']))}
            starcounts = {i: stat_data['star'].count(i) for i in list(set(stat_data['star']))}

            try:
                with open(myfile, 'a+') as f:
                    f.write(obslist + '\t' + str(way_int) + '\t' + starfiles[k][0:9] + '\t' + test[j][1] + '\t' + str(len(test[j][0])) + '\t' + str(round(tdelta,1)) + '\t' +  str(obsperhour) + '\n')
                    for m in range(len(list(obscounts.keys()))):
                        f.write(list(obscounts.keys())[m] + '\t' + str(list(obscounts.values())[m]) + '\t' + list(obscounts.keys())[m] + str(way_int) + '\n')
                    #for n in range(len(list(starcounts.keys()))):
                    #    f.write(list(starcounts.keys())[n] + '\t' + str(list(starcounts.values())[n]) + '\t' + str(way_int) + '\n')
                    f.close()
            except IndexError:
                pass

for obslist in obslist_files:
    stations = way(obslist)[1].split('\n')[0]
    way_int = 6 - stations.count('X')
    way_stat(obslist[0:10], logs_by_list[obslist[0:10]], 'way_observing_stats_2020.txt', way_int)

In [29]:
# Write observation information to file, now including the associated obslist and way
# UNDER DEVELOPMENT

global star_stat_file
star_stat_file = open('star_stat_file.txt', 'a+')
star_stat_file.write('Obslist\tWay\tUT Date\tStar\tScans of Star\n')
star_stat_file.close()

def star_stat(obslist, starfiles, myfile, way_int):
    for k in range(len(starfiles)):
        test = import_logs(starfiles[k])
        
        for j in range(len(test)):

            if test[j][1] == '':
                test[j][1] = 'XXXX'

            stat_data = {'obsnum': [test[j][0][i][0] for i in range(len(test[j][0]))],
                        'obstype': [test[j][0][i][1] for i in range(len(test[j][0]))],
                           'star': [test[j][0][i][2] for i in range(len(test[j][0]))],
                         'uttime': [test[j][0][i][3] for i in range(len(test[j][0]))]}

            FMT = '%H:%M:%S'

            starcounts = {i: stat_data['star'].count(i) for i in list(set(stat_data['star']))}

            try:
                with open(myfile, 'a+') as f:
                    f.write('\t\t' + starfiles[k][0:9] + '\n')
                    for n in range(len(list(starcounts.keys()))):
                        if list(starcounts.keys())[n] == 'FKV0000':
                            pass
                        else:
                            f.write('\t\t\t' + list(starcounts.keys())[n] + '\t' + str(list(starcounts.values())[n]) + '\n')
                    f.close()
            except IndexError:
                pass

for obslist in obslist_files:
    stations = way(obslist)[1].split('\n')[0]
    way_int = 6 - stations.count('X')
    star_stat(obslist[0:10], logs_by_list[obslist[0:10]], 'star_stat_file.txt', way_int)